In [30]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import warnings

## Import the CSV Data as Pandas DataFrame

In [2]:
df = pd.read_csv('cargurus.csv')

## Show Top 5 Records

In [4]:
df.head()

,Unnamed: 0,makeName,modelName,makeId,carYear,trimName,localizedTransmission,bodyTypeGroupId,bodyTypeName,mileage,exteriorColorName,normalizedExteriorColor,priceDifferential,daysOnMarket,dealRating,sellerId,listingPartnerId,sellerPostalCode,distance,serviceProviderId,serviceProviderName,localizedDriveTrain,localizedExteriorColor,localizedInteriorColor,sellerRating,reviewCount,howToShop,localizedFuelType,localizedDoors,driveTrain,localizedEngineDisplayName,ncapOverallSafetyRating,interiorColor
0,0,Jeep,Compass,m32,2021,Latitude 4WD,9-Speed Automatic,bg7,SUV / Crossover,23160.0,Diamond Black Crystal Pearlcoat,BLACK,-523.0,37,FAIR_PRICE,920924,52,15642,29.644321,286514,Jim Shorkey Chrysler Dodge Jeep Ram Fiat,Four-Wheel Drive,Diamond Black Crystal Pearlcoat,Black,4.333333,12.0,PICK_UP_ONLY,Gasoline,4 doors,FOUR_WHEEL_DRIVE,180 hp 2.4L I4,4,Black
1,1,RAM,1500,m191,2022,Big Horn Crew Cab 4WD,8-Speed Automatic,bg5,Pickup Truck,55327.0,Diamond Black Crystal Pearlcoat,BLACK,-1272.0,17,FAIR_PRICE,1361513,52,16117,39.720980,349314,Napleton Ellwood City Chrysler Dodge Jeep RAM,Four-Wheel Drive,Diamond Black Crystal Pearlcoat,Black,4.166667,5.0,PICK_UP_ONLY,Gasoline,4 doors,FOUR_WHEEL_DRIVE,305 hp 3.6L V6,Not Rated,Black
2,2,Chevrolet,Trailblazer,m1,2023,LT AWD,9-Speed Automatic,bg7,SUV / Crossover,29857.0,Mosaic Black Metallic,BLACK,-357.0,12,FAIR_PRICE,2211245,52,26059,20.927034,295982,Victory Toyota Highlands,Four-Wheel Drive,Mosaic Black Metallic,Black,NaN,NaN,PICK_UP_ONLY,Gasoline,4 doors,FOUR_WHEEL_DRIVE,155 hp 1.3L I3,Not Rated,Black
3,3,Ford,F-150,m2,2016,XL SuperCrew 4WD,6-Speed Automatic,bg5,Pickup Truck,106068.0,Race Red,RED,3356.0,2,GREAT_PRICE,1770263,58,15301,8.389171,294545,Washington Chevrolet,Four-Wheel Drive,Race Red,Dark Earth Gray,3.833333,6.0,HOME_DROP_OFF,Flex Fuel Vehicle,4 doors,FOUR_WHEEL_DRIVE,5L V8 Flex Fuel Vehicle,Not Rated,Dark Earth Gray
4,4,Honda,CR-V,m6,2013,LX AWD,5-Speed Automatic,bg7,SUV / Crossover,144150.0,Blue,BLUE,1774.0,106,GREAT_PRICE,2185824,24,16045,44.728123,280924,Lyndora Auto Sales,All-Wheel Drive,Blue,Gray,3.000000,2.0,PICK_UP_ONLY,Gasoline,4 doors,ALL_WHEEL_DRIVE,185 hp 2.4L I4,5,Gray


## drop Unnamed: 0  from dataframe

In [6]:
df = df.drop(columns=['Unnamed: 0'],axis=1)

## Preparing X and Y variables

In [66]:
X = df.drop(columns=['dealRating'],axis=1)

In [67]:
X.head()

,makeName,modelName,makeId,carYear,trimName,localizedTransmission,bodyTypeGroupId,bodyTypeName,mileage,exteriorColorName,normalizedExteriorColor,priceDifferential,daysOnMarket,sellerId,listingPartnerId,sellerPostalCode,distance,serviceProviderId,serviceProviderName,localizedDriveTrain,localizedExteriorColor,localizedInteriorColor,sellerRating,reviewCount,howToShop,localizedFuelType,localizedDoors,driveTrain,localizedEngineDisplayName,ncapOverallSafetyRating,interiorColor
0,Jeep,Compass,m32,2021,Latitude 4WD,9-Speed Automatic,bg7,SUV / Crossover,23160.0,Diamond Black Crystal Pearlcoat,BLACK,-523.0,37,920924,52,15642,29.644321,286514,Jim Shorkey Chrysler Dodge Jeep Ram Fiat,Four-Wheel Drive,Diamond Black Crystal Pearlcoat,Black,4.333333,12.0,PICK_UP_ONLY,Gasoline,4 doors,FOUR_WHEEL_DRIVE,180 hp 2.4L I4,4,Black
1,RAM,1500,m191,2022,Big Horn Crew Cab 4WD,8-Speed Automatic,bg5,Pickup Truck,55327.0,Diamond Black Crystal Pearlcoat,BLACK,-1272.0,17,1361513,52,16117,39.720980,349314,Napleton Ellwood City Chrysler Dodge Jeep RAM,Four-Wheel Drive,Diamond Black Crystal Pearlcoat,Black,4.166667,5.0,PICK_UP_ONLY,Gasoline,4 doors,FOUR_WHEEL_DRIVE,305 hp 3.6L V6,Not Rated,Black
2,Chevrolet,Trailblazer,m1,2023,LT AWD,9-Speed Automatic,bg7,SUV / Crossover,29857.0,Mosaic Black Metallic,BLACK,-357.0,12,2211245,52,26059,20.927034,295982,Victory Toyota Highlands,Four-Wheel Drive,Mosaic Black Metallic,Black,NaN,NaN,PICK_UP_ONLY,Gasoline,4 doors,FOUR_WHEEL_DRIVE,155 hp 1.3L I3,Not Rated,Black
3,Ford,F-150,m2,2016,XL SuperCrew 4WD,6-Speed Automatic,bg5,Pickup Truck,106068.0,Race Red,RED,3356.0,2,1770263,58,15301,8.389171,294545,Washington Chevrolet,Four-Wheel Drive,Race Red,Dark Earth Gray,3.833333,6.0,HOME_DROP_OFF,Flex Fuel Vehicle,4 doors,FOUR_WHEEL_DRIVE,5L V8 Flex Fuel Vehicle,Not Rated,Dark Earth Gray
4,Honda,CR-V,m6,2013,LX AWD,5-Speed Automatic,bg7,SUV / Crossover,144150.0,Blue,BLUE,1774.0,106,2185824,24,16045,44.728123,280924,Lyndora Auto Sales,All-Wheel Drive,Blue,Gray,3.000000,2.0,PICK_UP_ONLY,Gasoline,4 doors,ALL_WHEEL_DRIVE,185 hp 2.4L I4,5,Gray


In [68]:
y = df['dealRating'].values.reshape(-1, 1)

In [69]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
OneHot_Encoder = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", OneHot_Encoder, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [70]:
X = preprocessor.fit_transform(X)

In [71]:
X.shape

(1846, 2953)

In [72]:
# separate dataset into train and test

# Initialize LabelEncoder
label_encoder = LabelEncoder()



imputer = SimpleImputer(strategy='mean') 
imputer_1 = SimpleImputer(strategy='most_frequent')
# Use mean imputation, you can choose other strategies as w
X_imputed = imputer.fit_transform(X)
y_imputed = imputer_1.fit_transform(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_imputed,y_imputed,test_size=0.3,random_state=42)
# Fit and transform label encoder on the target variable
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
X_train.shape, X_test.shape

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


((1292, 2953), (554, 2953))

## Create an Evaluate Function to give all metrics after model Training

In [73]:
def evaluate_model(true, predicted):
    accuracy = accuracy_score(true, predicted)
    precision = precision_score(true, predicted, average='weighted')
    recall = recall_score(true, predicted, average='weighted')
    f1 = f1_score(true, predicted, average='weighted')
    cm = confusion_matrix(true, predicted)
    return accuracy, precision, recall, f1, cm


In [74]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(), 
    "CatBoost": CatBoostClassifier(verbose=False),
    "AdaBoost": AdaBoostClassifier()
}
model_list = []
r2_list =[]


# Now, train your model with the imputed data
model.fit(X_train, y_train_encoded)
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train_encoded) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_accuracy , _, _ ,_ ,_ = evaluate_model(y_train_encoded, y_train_pred)

    model_test_accuracy , _, _,_ ,_  = evaluate_model(y_test_encoded, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- model_train_accuracy: {:.4f}".format(model_train_accuracy))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- model_test_accuracy: {:.4f}".format(model_test_accuracy))
    r2_list.append(model_test_accuracy)
    
    print('='*35)
    print('\n')

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression
Model performance for Training set
- model_train_accuracy: 0.9899
----------------------------------
Model performance for Test set
- model_test_accuracy: 0.7347


Gradient Boosting
Model performance for Training set
- model_train_accuracy: 0.9946
----------------------------------
Model performance for Test set
- model_test_accuracy: 0.8412


Decision Tree
Model performance for Training set
- model_train_accuracy: 1.0000
----------------------------------
Model performance for Test set
- model_test_accuracy: 0.8032




C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest
Model performance for Training set
- model_train_accuracy: 1.0000
----------------------------------
Model performance for Test set
- model_test_accuracy: 0.6841


XGBoost
Model performance for Training set
- model_train_accuracy: 1.0000
----------------------------------
Model performance for Test set
- model_test_accuracy: 0.8466


CatBoost
Model performance for Training set
- model_train_accuracy: 0.9938
----------------------------------
Model performance for Test set
- model_test_accuracy: 0.8628


AdaBoost
Model performance for Training set
- model_train_accuracy: 0.6486
----------------------------------
Model performance for Test set
- model_test_accuracy: 0.6679




C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Results

In [75]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
5,CatBoost,0.862816
4,XGBoost,0.846570
1,Gradient Boosting,0.841155
2,Decision Tree,0.803249
0,Logistic Regression,0.734657
3,Random Forest,0.684116
6,AdaBoost,0.667870


## CatBoost

In [77]:
CatBoost = CatBoostClassifier(verbose=False)
CatBoost = CatBoost.fit(X_train, y_train)
y_pred = CatBoost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %accuracy)

 Accuracy of the model is 86.28
